In [51]:
import alpha_api
import pandas as pd
from sqlalchemy import create_engine
import datetime
import requests
from alpha_api import pgAdmin_pw

## Retrieve CSV Files

In [52]:
BTC_filepath = 'Resources/coin_Bitcoin.csv'
ETH_filepath = 'Resources/coin_Ethereum.csv'

## Store in Dataframes

Read csv files, drop unwanted columns, change date format

In [53]:
ETH_df = pd.read_csv(ETH_filepath)
ETH_df = ETH_df[['Date','Name','Symbol','Open','Close']]
ETH_df['Date'] = pd.to_datetime(ETH_df['Date'])
ETH_df['Date'] = ETH_df['Date'].dt.date
ETH_df.rename(columns = {'Date':'date','Name':'name','Symbol':'symbol','Open':'open','Close':'close'}, inplace=True)
ETH_df.head()

,date,name,symbol,open,close
0,2015-08-08,Ethereum,ETH,2.793760,0.753325
1,2015-08-09,Ethereum,ETH,0.706136,0.701897
2,2015-08-10,Ethereum,ETH,0.713989,0.708448
3,2015-08-11,Ethereum,ETH,0.708087,1.067860
4,2015-08-12,Ethereum,ETH,1.058750,1.217440


In [54]:
BTC_df = pd.read_csv(BTC_filepath)
BTC_df = BTC_df[['Date','Name','Symbol','Open','Close']]
BTC_df['Date'] = pd.to_datetime(BTC_df['Date'])
BTC_df['Date'] = BTC_df['Date'].dt.date
BTC_df.rename(columns = {'Date':'date','Name':'name','Symbol':'symbol','Open':'open','Close':'close'}, inplace=True)
BTC_df

,date,name,symbol,open,close
0,2013-04-29,Bitcoin,BTC,134.444000,144.539993
1,2013-04-30,Bitcoin,BTC,144.000000,139.000000
2,2013-05-01,Bitcoin,BTC,139.000000,116.989998
3,2013-05-02,Bitcoin,BTC,116.379997,105.209999
4,2013-05-03,Bitcoin,BTC,106.250000,97.750000
...,...,...,...,...,...
2986,2021-07-02,Bitcoin,BTC,33549.600177,33897.048590
2987,2021-07-03,Bitcoin,BTC,33854.421362,34668.548402
2988,2021-07-04,Bitcoin,BTC,34665.564866,35287.779766
2989,2021-07-05,Bitcoin,BTC,35284.344430,33746.002456


In [55]:
BTC_df_dates = BTC_df.loc[BTC_df['date'] >= datetime.date(2015,8,8)]
BTC_df_dates

,date,name,symbol,open,close
831,2015-08-08,Bitcoin,BTC,279.742004,260.997009
832,2015-08-09,Bitcoin,BTC,261.115997,265.083008
833,2015-08-10,Bitcoin,BTC,265.477997,264.470001
834,2015-08-11,Bitcoin,BTC,264.342010,270.385986
835,2015-08-12,Bitcoin,BTC,270.597992,266.376007
...,...,...,...,...,...
2986,2021-07-02,Bitcoin,BTC,33549.600177,33897.048590
2987,2021-07-03,Bitcoin,BTC,33854.421362,34668.548402
2988,2021-07-04,Bitcoin,BTC,34665.564866,35287.779766
2989,2021-07-05,Bitcoin,BTC,35284.344430,33746.002456


## Retrieve JSON data

In [56]:
eth_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=ETH&market=USD&apikey={alpha_api}'
btc_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={alpha_api}'

In [57]:
ETH_response = requests.get(eth_url).json()
ETH_dates = ETH_response['Time Series (Digital Currency Daily)']
ETH_dates

{'2022-08-19': {'1a. open (USD)': '1846.39000000',
  '1b. open (USD)': '1846.39000000',
  '2a. high (USD)': '1848.00000000',
  '2b. high (USD)': '1848.00000000',
  '3a. low (USD)': '1811.10000000',
  '3b. low (USD)': '1811.10000000',
  '4a. close (USD)': '1819.00000000',
  '4b. close (USD)': '1819.00000000',
  '5. volume': '52316.17290000',
  '6. market cap (USD)': '52316.17290000'},
 '2022-08-18': {'1a. open (USD)': '1834.24000000',
  '1b. open (USD)': '1834.24000000',
  '2a. high (USD)': '1882.37000000',
  '2b. high (USD)': '1882.37000000',
  '3a. low (USD)': '1821.25000000',
  '3b. low (USD)': '1821.25000000',
  '4a. close (USD)': '1846.39000000',
  '4b. close (USD)': '1846.39000000',
  '5. volume': '606254.02400000',
  '6. market cap (USD)': '606254.02400000'},
 '2022-08-17': {'1a. open (USD)': '1876.67000000',
  '1b. open (USD)': '1876.67000000',
  '2a. high (USD)': '1959.43000000',
  '2b. high (USD)': '1959.43000000',
  '3a. low (USD)': '1818.74000000',
  '3b. low (USD)': '1818.7

In [58]:
ETH_openlist = []
ETH_closelist = []
ETH_datelist = []
for date in ETH_dates:
    ETH_datelist.append(date)
    ETH_openlist.append(ETH_dates[date]['1a. open (USD)'])
    ETH_closelist.append(ETH_dates[date]['4a. close (USD)'])

ETH_dict = {'date':ETH_datelist,
    'open':ETH_openlist,
    'close':ETH_closelist}

ETH_dataframe = pd.DataFrame(ETH_dict)
ETH_dataframe

,date,open,close
0,2022-08-19,1846.39000000,1819.00000000
1,2022-08-18,1834.24000000,1846.39000000
2,2022-08-17,1876.67000000,1834.25000000
3,2022-08-16,1899.05000000,1876.67000000
4,2022-08-15,1935.32000000,1899.06000000
...,...,...,...
995,2019-11-28,152.61000000,150.72000000
996,2019-11-27,147.47000000,152.62000000
997,2019-11-26,145.81000000,147.47000000
998,2019-11-25,139.99000000,145.69000000


In [59]:
BTC_response = requests.get(btc_url).json()
BTC_dates = BTC_response['Time Series (Digital Currency Daily)']
BTC_dates

{'2022-08-19': {'1a. open (USD)': '23191.45000000',
  '1b. open (USD)': '23191.45000000',
  '2a. high (USD)': '23208.67000000',
  '2b. high (USD)': '23208.67000000',
  '3a. low (USD)': '22877.00000000',
  '3b. low (USD)': '22877.00000000',
  '4a. close (USD)': '22925.80000000',
  '4b. close (USD)': '22925.80000000',
  '5. volume': '11716.88274000',
  '6. market cap (USD)': '11716.88274000'},
 '2022-08-18': {'1a. open (USD)': '23342.66000000',
  '1b. open (USD)': '23342.66000000',
  '2a. high (USD)': '23600.00000000',
  '2b. high (USD)': '23600.00000000',
  '3a. low (USD)': '23111.04000000',
  '3b. low (USD)': '23111.04000000',
  '4a. close (USD)': '23191.20000000',
  '4b. close (USD)': '23191.20000000',
  '5. volume': '144185.97011000',
  '6. market cap (USD)': '144185.97011000'},
 '2022-08-17': {'1a. open (USD)': '23856.15000000',
  '1b. open (USD)': '23856.15000000',
  '2a. high (USD)': '24446.71000000',
  '2b. high (USD)': '24446.71000000',
  '3a. low (USD)': '23180.40000000',
  '3b

In [60]:
BTC_openlist = []
BTC_closelist = []
BTC_datelist = []
for date in BTC_dates:
    BTC_datelist.append(date)
    BTC_openlist.append(BTC_dates[date]['1a. open (USD)'])
    BTC_closelist.append(BTC_dates[date]['4a. close (USD)'])

BTC_dict = {'date':BTC_datelist,
    'open':BTC_openlist,
    'close':BTC_closelist}

BTC_dataframe = pd.DataFrame(BTC_dict)
BTC_dataframe

,date,open,close
0,2022-08-19,23191.45000000,22925.80000000
1,2022-08-18,23342.66000000,23191.20000000
2,2022-08-17,23856.15000000,23342.66000000
3,2022-08-16,24093.04000000,23854.74000000
4,2022-08-15,24305.25000000,24094.82000000
...,...,...,...
995,2019-11-28,7507.90000000,7419.49000000
996,2019-11-27,7154.75000000,7508.52000000
997,2019-11-26,7109.99000000,7156.14000000
998,2019-11-25,6900.23000000,7109.57000000


In [61]:
BTC_symbol = 'BTC'
BTC_name = 'Bitcoin'
ETH_symbol = 'ETH'
ETH_name = 'Ethereum'

BTC_dataframe['name'] = BTC_name
BTC_dataframe['symbol'] = BTC_symbol
ETH_dataframe['name'] = ETH_name
ETH_dataframe['symbol'] = ETH_symbol
ETH_dataframe = ETH_dataframe[['date','name','symbol','open','close']]

BTC_dataframe = BTC_dataframe[['date','name','symbol','open','close']]


BTC_dataframe['date'] = pd.to_datetime(BTC_dataframe['date'])
BTC_dataframe['date'] = BTC_dataframe['date'].dt.date
BTC_dataframe = BTC_dataframe.loc[BTC_dataframe['date'] >= datetime.date(2021,7,7)]
BTC_dataframe.sort_values('date', ascending=True, inplace=True)

In [62]:
ETH_dataframe['date'] = pd.to_datetime(ETH_dataframe['date'])
ETH_dataframe['date'] = ETH_dataframe['date'].dt.date
ETH_dataframe = ETH_dataframe.loc[ETH_dataframe['date'] >= datetime.date(2021,7,7)]
ETH_dataframe.sort_values('date', ascending=True, inplace=True)
ETH_dataframe

,date,name,symbol,open,close
408,2021-07-07,Ethereum,ETH,2322.51000000,2316.57000000
407,2021-07-08,Ethereum,ETH,2316.25000000,2116.33000000
406,2021-07-09,Ethereum,ETH,2116.32000000,2146.99000000
405,2021-07-10,Ethereum,ETH,2146.99000000,2110.69000000
404,2021-07-11,Ethereum,ETH,2110.68000000,2140.45000000
...,...,...,...,...,...
4,2022-08-15,Ethereum,ETH,1935.32000000,1899.06000000
3,2022-08-16,Ethereum,ETH,1899.05000000,1876.67000000
2,2022-08-17,Ethereum,ETH,1876.67000000,1834.25000000
1,2022-08-18,Ethereum,ETH,1834.24000000,1846.39000000


In [63]:
ETH_dataframe

,date,name,symbol,open,close
408,2021-07-07,Ethereum,ETH,2322.51000000,2316.57000000
407,2021-07-08,Ethereum,ETH,2316.25000000,2116.33000000
406,2021-07-09,Ethereum,ETH,2116.32000000,2146.99000000
405,2021-07-10,Ethereum,ETH,2146.99000000,2110.69000000
404,2021-07-11,Ethereum,ETH,2110.68000000,2140.45000000
...,...,...,...,...,...
4,2022-08-15,Ethereum,ETH,1935.32000000,1899.06000000
3,2022-08-16,Ethereum,ETH,1899.05000000,1876.67000000
2,2022-08-17,Ethereum,ETH,1876.67000000,1834.25000000
1,2022-08-18,Ethereum,ETH,1834.24000000,1846.39000000


Append Dataframes

In [64]:
Full_BTC = BTC_df_dates.append(BTC_dataframe)
Full_BTC

,date,name,symbol,open,close
831,2015-08-08,Bitcoin,BTC,279.742,260.997
832,2015-08-09,Bitcoin,BTC,261.116,265.083
833,2015-08-10,Bitcoin,BTC,265.478,264.47
834,2015-08-11,Bitcoin,BTC,264.342,270.386
835,2015-08-12,Bitcoin,BTC,270.598,266.376
...,...,...,...,...,...
4,2022-08-15,Bitcoin,BTC,24305.25000000,24094.82000000
3,2022-08-16,Bitcoin,BTC,24093.04000000,23854.74000000
2,2022-08-17,Bitcoin,BTC,23856.15000000,23342.66000000
1,2022-08-18,Bitcoin,BTC,23342.66000000,23191.20000000


In [65]:
Full_ETH = ETH_df.append(ETH_dataframe)
Full_ETH

,date,name,symbol,open,close
0,2015-08-08,Ethereum,ETH,2.79376,0.753325
1,2015-08-09,Ethereum,ETH,0.706136,0.701897
2,2015-08-10,Ethereum,ETH,0.713989,0.708448
3,2015-08-11,Ethereum,ETH,0.708087,1.06786
4,2015-08-12,Ethereum,ETH,1.05875,1.21744
...,...,...,...,...,...
4,2022-08-15,Ethereum,ETH,1935.32000000,1899.06000000
3,2022-08-16,Ethereum,ETH,1899.05000000,1876.67000000
2,2022-08-17,Ethereum,ETH,1876.67000000,1834.25000000
1,2022-08-18,Ethereum,ETH,1834.24000000,1846.39000000


Connect to pgAdmin

In [66]:
protocol = 'postgresql'
username = 'postgres'
password = f'{pgAdmin_pw}'
host = 'localhost'
port = 5432
database_name = 'crypto_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [67]:
engine.table_names()


['btc', 'eth']

In [68]:
Full_ETH.to_sql(name='eth', con=engine, if_exists='append', index=False)

In [69]:
Full_BTC.to_sql(name='btc', con=engine, if_exists='append', index=False)

In [70]:
pd.read_sql_query('select * from eth', con=engine).head()


,date,name,symbol,open,close
0,2015-08-08,Ethereum,ETH,2.793760,0.753325
1,2015-08-09,Ethereum,ETH,0.706136,0.701897
2,2015-08-10,Ethereum,ETH,0.713989,0.708448
3,2015-08-11,Ethereum,ETH,0.708087,1.067860
4,2015-08-12,Ethereum,ETH,1.058750,1.217440


In [71]:
pd.read_sql_query('select * from btc', con=engine).head()

,date,name,symbol,open,close
0,2015-08-08,Bitcoin,BTC,279.742004,260.997009
1,2015-08-09,Bitcoin,BTC,261.115997,265.083008
2,2015-08-10,Bitcoin,BTC,265.477997,264.470001
3,2015-08-11,Bitcoin,BTC,264.342010,270.385986
4,2015-08-12,Bitcoin,BTC,270.597992,266.376007


In [72]:
pd.read_sql_query('select * from btc_eth', con=engine).head()

,date,btc_open,btc_close,eth_open,eth_close
0,2015-08-08,279.742004,260.997009,2.793760,0.753325
1,2015-08-09,261.115997,265.083008,0.706136,0.701897
2,2015-08-10,265.477997,264.470001,0.713989,0.708448
3,2015-08-11,264.342010,270.385986,0.708087,1.067860
4,2015-08-12,270.597992,266.376007,1.058750,1.217440
